In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import ExtraTreesRegressor
import xgboost as xgb

import sys
sys.path.append('../')

from src.functions import Functions
from src.modeling import Modeling

import warnings
warnings.filterwarnings('ignore')

In [2]:
features = pd.read_csv('../data/raw_data/features.csv')
sales_train = pd.read_csv('../data/raw_data/train.csv')
stores = pd.read_csv('../data/raw_data/stores.csv')
sales_test = pd.read_csv('../data/raw_data/test.csv')

In [3]:
features = Functions.rollingInterpolation(features)

In [4]:
df_all_train = sales_train.merge(features, how="left", on=["Store","Date","IsHoliday"]).merge(stores, how="left", on=["Store"])
df_all_test = sales_test.merge(features, how="left", on=["Store","Date","IsHoliday"]).merge(stores, how="left", on=["Store"])

df_all_train['train_or_test'] = 'train'
df_all_test['train_or_test'] = 'test'
merged_df = pd.concat([df_all_train, df_all_test], sort=False)

merged_df = merged_df.sort_values(by=['Store','Dept','Date'], axis=0).reset_index().drop(columns='index')

In [5]:
merged_df = pd.get_dummies(merged_df, columns=["Type"])

LE = LabelEncoder()
merged_df['IsHoliday'] = LE.fit_transform(merged_df['IsHoliday'])

In [6]:
merged_df = Functions.addDateFeature(merged_df)

In [7]:
#merged_df =  Functions.createDateStatsFeatures(merged_df, 'Monthly_Sales_', ['Store','Dept', 'Month', 'Size'])

In [8]:
def createDateStatsFeatures(df, variable, gp_cols, target='Weekly_Sales', funcs={'mean':np.mean,
                                                                                 'median':np.median,
                                                                                 'max':np.max,
                                                                                 'min':np.min,
                                                                                 'std':np.std}):
        train_df = df.loc[~(df.train_or_test=='test'), :]
        gp = train_df.groupby(gp_cols)
        newdf = df[gp_cols].drop_duplicates().reset_index(drop=True)
        for name, func in funcs.items():
            tmp = gp[target].agg(func).reset_index()
            tmp.rename(columns={target:variable + name}, inplace=True)
            newdf = newdf.merge(tmp, on=gp_cols, how='left')
        return df.merge(newdf, on=gp_cols, how='left')

In [9]:
#df = merged_df
#gp_cols = ['Store','Dept', 'Month', 'Size']
#target='Weekly_Sales'
#variable = 'a'
#funcs={'mean':np.mean,
#                                                                                 'median':np.median,
#                                                                                 'max':np.max,
#                                                                                 'min':np.min,
#                                                                                 'std':np.std}
#
#train_df = df.loc[~(df.train_or_test=='test'), :]
#gp = train_df.groupby(gp_cols)
#newdf = df[gp_cols].drop_duplicates().reset_index(drop=True)

In [10]:
#for name, func in funcs.items():
#    tmp = gp[target].agg(func).reset_index()
#    tmp.rename(columns={target:variable + name}, inplace=True)
#    newdf = newdf.merge(tmp, on=gp_cols, how='left')

In [11]:
merged_df =  Functions.createDateStatsFeatures(merged_df, 'Monthly_Sales_', ['Store','Dept', 'Month'])
merged_df =  Functions.createDateStatsFeatures(merged_df, 'Daily_Sales_', ['Store','Dept', 'Day'])
merged_df =  Functions.createDateStatsFeatures(merged_df, 'WeekofMonth_Sales_', ['Store','Dept', 'WeekofMonth'])
merged_df =  Functions.createDateStatsFeatures(merged_df, 'WeekofYear_Sales_', ['Store','Dept', 'WeekofYear'])

In [12]:
merged_df =  Functions.createDateStatsFeatures(merged_df, 'Size_Sales_', ['Store','Dept', 'Size'])
merged_df =  Functions.createDateStatsFeatures(merged_df, 'Unemployment_Sales_', ['Store','Dept', 'Unemployment'])
merged_df =  Functions.createDateStatsFeatures(merged_df, 'IsHoliday_Sales_', ['Store','Dept', 'IsHoliday'])

merged_df =  Functions.createDateStatsFeatures(merged_df, 'Size_Month_Sales_', ['Store','Dept', 'Size', 'Month'])
merged_df =  Functions.createDateStatsFeatures(merged_df, 'Unemployment_Month_Sales_', ['Store','Dept', 'Unemployment','Month'])
merged_df =  Functions.createDateStatsFeatures(merged_df, 'IsHoliday_Month_Sales_', ['Store','Dept', 'IsHoliday','Month'])
merged_df =  Functions.createDateStatsFeatures(merged_df, 'IsHoliday_Month_Size_Sales_', ['Store','Dept', 'IsHoliday','Month', 'Size'])

merged_df =  Functions.createDateStatsFeatures(merged_df, 'Size_WeekofMonth_Sales_', ['Store','Dept', 'Size', 'WeekofMonth'])
merged_df =  Functions.createDateStatsFeatures(merged_df, 'Unemployment_WeekofMonth_Sales_', ['Store','Dept', 'Unemployment','WeekofMonth'])
merged_df =  Functions.createDateStatsFeatures(merged_df, 'IsHoliday_WeekofMonth_Sales_', ['Store','Dept', 'IsHoliday','WeekofMonth'])

merged_df =  Functions.createDateStatsFeatures(merged_df, 'Size_WeekofYear_Sales_', ['Store','Dept', 'Size', 'WeekofYear'])
merged_df =  Functions.createDateStatsFeatures(merged_df, 'Unemployment_WeekofYear_Sales_', ['Store','Dept', 'Unemployment','WeekofYear'])
merged_df =  Functions.createDateStatsFeatures(merged_df, 'IsHoliday_WeekofYear_Sales_', ['Store','Dept', 'IsHoliday','WeekofYear'])

merged_df =  Functions.createDateStatsFeatures(merged_df, 'Month_Day_Sales_', ['Store','Dept', 'Month', 'Day'])
merged_df =  Functions.createDateStatsFeatures(merged_df, 'Month_WeekofMonth_Sales_', ['Store','Dept', 'Month', 'WeekofMonth'])
merged_df =  Functions.createDateStatsFeatures(merged_df, 'Month_WeekofYear_Sales_', ['Store','Dept', 'Month', 'WeekofYear'])
merged_df =  Functions.createDateStatsFeatures(merged_df, 'Day_WeekofMonth_Sales_', ['Store','Dept', 'Day', 'WeekofMonth'])
merged_df =  Functions.createDateStatsFeatures(merged_df, 'WeekofMonth_WeekofYear_Sales_', ['Store','Dept', 'WeekofMonth', 'WeekofYear'])

merged_df =  Functions.createDateStatsFeatures(merged_df, 'Month_WeekofMonth_WeekofMonth_Sales_', ['Store','Dept', 'Month', 'WeekofMonth', 'WeekofYear'])

In [13]:
pd.DataFrame(merged_df.loc[0, :]).to_csv('./df.csv')

In [14]:
merged_df.CPI

0         211.096358
1         211.242170
2         211.289143
3         211.319643
4         211.350143
             ...    
536629    193.484651
536630    193.516047
536631    193.545350
536632    193.570990
536633    193.589304
Name: CPI, Length: 536634, dtype: float64